# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT
    column_1,
    aggregate_function(column_2)
FROM
    tbl_name
GROUP BY
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami.

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

Konfiguracja połączenia z bazą danych.

In [264]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:

1. Znajdź listę wszystkich filmów o tej samej długości.

In [265]:
pd.read_sql('SELECT title, length FROM film', con=connection).pivot(index=['length', 'title'], columns=[]).sort_index()

Empty DataFrame
Columns: []
Index: [(46, Alien Center), (46, Iron Moon), (46, Kwai Homeward), (46, Labyrinth League), (46, Ridgemont Submarine), (47, Divorce Shining), (47, Downhill Enough), (47, Halloween Nuts), (47, Hanover Galaxy), (47, Hawk Chill), (47, Shanghai Tycoon), (47, Suspects Quills), (48, Ace Goldfinger), (48, Heaven Freedom), (48, Midsummer Groundhog), (48, Notting Speakeasy), (48, Odds Boogie), (48, Paradise Sabrina), (48, Pelican Comforts), (48, Rush Goodfellas), (48, Stepmom Dream), (48, Sunset Racer), (48, Valentine Vanishing), (49, Doors President), (49, Grosse Wonderful), (49, Heavenly Gun), (49, Hook Chariots), (49, Hurricane Affair), (50, Adaptation Holes), (50, Blues Instinct), (50, Crossing Divorce), (50, Lion Uncut), (50, Muppet Mile), (50, Natural Stock), (50, Pilot Hoosiers), (50, Smoking Barbarella), (50, Zorro Ark), (51, Champion Flatliners), (51, Deep Crusade), (51, English Bulworth), (51, Excitement Eve), (51, Frisco Forrest), (51, Hall Cassidy), (51, Simon North), (52, Caddyshack Jedi), (52, Harper Dying), (52, Lust Lock), (52, Side Ark), (52, Spartacus Cheaper), (52, Trojan Tomorrow), (52, Westward Seabiscuit), (53, Beneath Rush), (53, Cabin Flash), (53, Gump Date), (53, Magnificent Chitty), (53, Movie Shakespeare), (53, Primary Glass), (53, Summer Scarface), (53, Tequila Past), (53, Thin Sagebrush), (54, Airport Pollock), (54, Go Purple), (54, Juggler Hardly), (54, Kill Brotherhood), (54, October Submarine), (54, Sense Greek), (55, Coast Rainbow), (55, Wolves Desire), (56, Bride Intrigue), (56, Cupboard Sinners), (56, Destiny Saturday), (56, Goodfellas Salute), (56, Matrix Snowman), (57, Alter Victory), (57, Cranes Reservoir), (57, Dawn Pond), (57, Doctor Grail), (57, Mosquito Armageddon), (57, Noon Papi), (57, Storm Happiness), (58, Closer Bang), (58, Dances None), (58, Fantasy Troopers), (58, Jekyll Frogmen), (58, Oklahoma Jumanji), (58, Rings Heartbreakers), (58, Super Wyoming), (59, Commandments Express), (59, Daughter Madigan), (59, Heartbreakers Bright), (59, Interview Liaisons), (59, Jet Neighbors), (59, Legend Jedi), (59, Minority Kiss), (59, Vision Torque), (59, Whisperer Giant), (60, Bubble Grosse), (60, Jersey Sassy), (60, Mockingbird Hollywood), (60, Phantom Glory), ...]

[1000 rows x 0 columns]

2. Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [266]:
pd.read_sql('SELECT c.first_name, c.last_name, c2.city FROM customer c LEFT JOIN address a ON c.address_id = a.address_id '
            'LEFT JOIN city c2 ON a.city_id = c2.city_id', con=connection).groupby('city').filter(lambda customers: len(customers) > 1).pivot(index=['city', 'first_name', 'last_name'], columns=[]).sort_index()

Empty DataFrame
Columns: []
Index: [(Aurora, Clinton, Buford), (Aurora, Scott, Shelley), (London, Cecil, Vines), (London, Mattie, Hoffman)]

Jak się okazuje klienci

3. Oblicz średni koszt wypożyczenia wszystkich filmów.

In [267]:
pd.read_sql('SELECT f.title, AVG(f.rental_rate) '
            'FROM film f '
            'GROUP BY f.title', con=connection)

,title,avg
0,Frontier Cabin,4.99
1,Arachnophobia Rollercoaster,2.99
2,Cruelty Unforgiven,0.99
3,Intolerable Intentions,4.99
4,Center Dinosaur,4.99
...,...,...
995,Luke Mummy,2.99
996,Spy Mile,2.99
997,Tramp Others,0.99
998,Butterfly Chocolat,0.99


4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [268]:
pd.read_sql('SELECT c.name as category, COUNT(f.title) AS films_in_category FROM film f LEFT JOIN film_category fc ON f.film_id = fc.film_id LEFT JOIN category c ON fc.category_id = c.category_id GROUP BY c.name', con=connection)

,category,films_in_category
0,Horror,56
1,Comedy,58
2,Sci-Fi,61
3,Drama,62
4,Foreign,73
5,Classics,57
6,Games,61
7,New,63
8,Travel,57
9,Music,51


5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [269]:
pd.read_sql('SELECT country.country AS country, COUNT(c.customer_id) AS customer_count FROM customer c INNER JOIN address a ON c.address_id = a.address_id INNER JOIN city ON  city.city_id = a.city_id INNER JOIN country ON city.country_id = country.country_id GROUP BY country.country', con=connection)

,country,customer_count
0,Cambodia,2
1,Turkey,15
2,Germany,7
3,Madagascar,1
4,Chad,1
...,...,...
103,Sudan,2
104,United Arab Emirates,3
105,Tanzania,3
106,French Polynesia,2


6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [270]:
pd.read_sql('SELECT * FROM store WHERE store_id IN (SELECT store_id  FROM customer GROUP BY store_id HAVING (COUNT(store_id) > 100 AND COUNT(store_id) < 300))', con=connection)

,store_id,manager_staff_id,address_id,last_update
0,2,2,2,2006-02-15 09:57:12


7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [271]:
# zakładam że każde wypożyczenie liczy się jako obejrzenie tego filmu jeden raz przez danego klienta
pd.read_sql('SELECT r.customer_id FROM rental r INNER JOIN customer c ON r.customer_id = c.customer_id RIGHT JOIN inventory i ON r.inventory_id = i.inventory_id INNER JOIN film f ON i.film_id = f.film_id GROUP BY r.customer_id HAVING SUM(f.length) / 60 > 200', con=connection)
# i wychodzi na to że z tym założeniem nikt nie oglądał filmów przez 200h

,customer_id


8. Oblicz średnią wartość wypożyczenia filmu.

In [272]:
pd.read_sql('SELECT AVG(rental_rate) AS avg_film_rental_rate FROM film', con=connection)

,avg_film_rental_rate
0,2.98


9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [273]:
pd.read_sql('SELECT c.name as category, AVG(f.length) AS avg_length FROM film f LEFT JOIN film_category fc ON f.film_id = fc.film_id LEFT JOIN category c ON fc.category_id = c.category_id GROUP BY c.name', con=connection)

,category,avg_length
0,Horror,112.482143
1,Comedy,115.827586
2,Sci-Fi,108.196721
3,Drama,120.838710
4,Foreign,121.698630
5,Classics,111.666667
6,Games,127.836066
7,New,111.126984
8,Travel,113.315789
9,Music,113.647059


10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [274]:
pd.read_sql('SELECT * '
            'INTO TEMPORARY lt_temp '
            'FROM '
            '(SELECT c.category_id, c.name as category_name, MAX(LENGTH(f.title)) AS longest_title_len '
            'FROM film f '
            'INNER JOIN film_category fc ON f.film_id = fc.film_id '
            'INNER JOIN category c ON fc.category_id = c.category_id '
            'GROUP BY c.category_id, c.name) AS foo; '
            'SELECT lt.category_name as category, f.title as longest_title, LENGTH(f.title) as title_len '
            'FROM film f '
            'INNER JOIN film_category fc ON fc.film_id = f.film_id '
            'INNER JOIN lt_temp lt ON fc.category_id = lt.category_id '
            'WHERE LENGTH(f.title) = lt.longest_title_len', con=connection).pivot(index=['category', 'title_len', 'longest_title'], columns=[]).sort_index()
# bardzo pokręcony sposób, ale chyba działa

Empty DataFrame
Columns: []
Index: [(Action, 23, Entrapment Satisfaction), (Animation, 22, Telemark Heartbreakers), (Children, 20, Heartbreakers Bright), (Children, 20, Microcosmos Paradise), (Children, 20, Sweethearts Suspects), (Classics, 23, Extraordinary Conquerer), (Comedy, 23, Trainspotting Strangers), (Documentary, 22, Deliverance Mulholland), (Documentary, 22, Intolerable Intentions), (Drama, 22, Goldfinger Sensibility), (Family, 22, Resurrection Silverado), (Foreign, 20, Ballroom Mockingbird), (Foreign, 20, Impossible Prejudice), (Foreign, 20, Reservoir Adaptation), (Games, 21, Bulworth Commandments), (Games, 21, Creatures Shakespeare), (Horror, 27, Arachnophobia Rollercoaster), (Music, 22, Confidential Interview), (New, 21, Flamingos Connecticut), (Sci-Fi, 20, Barbarella Streetcar), (Sci-Fi, 20, Cincinatti Whisperer), (Sci-Fi, 20, Silverado Goldfinger), (Sci-Fi, 20, Unforgiven Zoolander), (Sci-Fi, 20, Wonderland Christmas), (Sports, 25, Satisfaction Confidential), (Travel, 23, Desperate Trainspotting)]

11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [275]:
pd.read_sql('SELECT * '
            'INTO TEMPORARY lf_temp '
            'FROM '
            '(SELECT c.category_id, c.name as category_name, MAX(f.length) AS longest_film '
            'FROM film f '
            'LEFT JOIN film_category fc ON f.film_id = fc.film_id '
            'LEFT JOIN category c ON fc.category_id = c.category_id '
            'GROUP BY c.category_id, c.name) AS foo; '
            'SELECT lf.category_name as category, f.title as longest_film, f.length as length_minutes '
            'FROM film f '
            'INNER JOIN film_category fc ON fc.film_id = f.film_id '
            'INNER JOIN lf_temp lf ON fc.category_id = lf.category_id '
            'WHERE f.length = lf.longest_film', con=connection).pivot(index=['category', 'length_minutes', 'longest_film'], columns=[]).sort_index()
# bardzo pokręcony sposób, ale chyba działa

Empty DataFrame
Columns: []
Index: [(Action, 185, Darn Forrester), (Action, 185, Worst Banger), (Animation, 185, Gangs Pride), (Animation, 185, Pond Seattle), (Children, 178, Fury Murder), (Children, 178, Wrong Behavior), (Classics, 184, Conspiracy Spirit), (Comedy, 185, Control Anthem), (Documentary, 183, Wife Turn), (Documentary, 183, Young Language), (Drama, 181, Jacket Frisco), (Family, 184, King Evolution), (Foreign, 184, Crystal Breaking), (Foreign, 184, Sorority Queen), (Games, 185, Chicago North), (Horror, 181, Analyze Hoosiers), (Horror, 181, Love Suicides), (Music, 185, Home Pity), (New, 183, Frontier Cabin), (Sci-Fi, 185, Soldiers Evolution), (Sports, 184, Smoochy Control), (Travel, 185, Muscle Bright), (Travel, 185, Sweet Brotherhood)]

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [286]:
import main

main.film_in_category(category_id=7)

In [287]:
main.number_films_in_category(category_id=5)

In [278]:
main.number_film_by_length(min_length=70, max_length=120)

In [279]:
main.client_from_city(city='Warsaw')

In [280]:
main.avg_amount_by_length(length=88)

In [281]:
main.client_by_sum_length(sum_min=1500)

In [282]:
main.category_statistic_length(name='Comedy')